In [63]:
import json
import random
import pandas as pd
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setAppName("BDA").setMaster("local")
sc = SparkContext(conf=conf)

In [65]:
def addRandomMonths():
    random_count=[]
    months=['Jan','Feb','Mar','Apr','May','Jun','Jul', 'Aug','Sep','Oct','Nov','Dec']
    for i in range(len(months)):
        item=(months[i],random.randint(1,101))
        random_count.append(item)
    return random_count      

In [16]:
places_path="file:///Users/ritukaushik/Downloads/TouristPlaces.xlsx"
dfs = pd.read_excel(places_path, sheetname=None)
places=list(dfs.items())
rdd_places=sc.parallelize(places)
tourist_spots=rdd_places.map(lambda x: (x[0],(x[1].values)))
ts=tourist_spots.flatMap(lambda x:[(x[1][i], x[0]) for i in range(len(x[1]))])
ts_formatted=ts.map(lambda x: (x[0][0], (x[0][1],x[0][2],x[1])))
ts_info=ts_formatted.filter(lambda x: str(x[0]).strip() not in ('nan', 'Name'))

In [17]:
ts_info = ts_info.map(lambda x: (''.join(x[0].split()).lower(),x[1]))

In [18]:
ts_info.collect()

[('onedirection', ('Museums', 'Indianapolis', 'Indiana')),
 ('fortwanechildrenzoo', ('Zoos', 'Fort Wayne', 'Indiana')),
 ('basilicaofthedacredheart', ('Church', 'South Bend', 'Indiana')),
 ('universityofnotredame', ('University', 'South Bend', 'Indiana')),
 ('parkviewfield', ('Arenas', 'Fort Wayne', 'Indiana')),
 ('dreamcarmuseum', ('Museums', 'Evansville', 'Indiana')),
 ('victoryfield', ('Stadiums', 'Indianapolis', 'Indiana')),
 ('auburncordduesenbergautomobilemuseum', ('Museums', 'Auburn', 'Indiana')),
 ('bluegatetheatre', ('Theaters', 'Shipshewana', 'Indiana')),
 ('holidayworldsplashinsafari', ('Theme Parks', 'Santa Claus', 'Indiana')),
 ('indianapolismotorspeedwaymuseum', ('Museums', 'Indianapolis', 'Indiana')),
 ('indianauniversity', ('Educational Sites', 'Bloomington', 'Indiana')),
 ('indianapolismuseumofart', ('Historic Sites', 'Indianapolis', 'Indiana')),
 ('lucasoilstadium', ('Stadiums', 'Indianapolis', 'Indiana')),
 ('allencountypubliclibrary', ('Libraries', 'Fort Wayne', 'In

In [68]:
ts_months=ts_info.map(lambda x: (x[0],x[1], addRandomMonths()))

In [84]:
ts_months.take(2)

[('onedirection',
  ('Museums', 'Indianapolis', 'Indiana'),
  [('Jan', 84),
   ('Feb', 87),
   ('Mar', 90),
   ('Apr', 86),
   ('May', 100),
   ('Jun', 54),
   ('Jul', 84),
   ('Aug', 29),
   ('Sep', 5),
   ('Oct', 85),
   ('Nov', 13),
   ('Dec', 1)]),
 ('fortwanechildrenzoo',
  ('Zoos', 'Fort Wayne', 'Indiana'),
  [('Jan', 16),
   ('Feb', 56),
   ('Mar', 49),
   ('Apr', 87),
   ('May', 51),
   ('Jun', 66),
   ('Jul', 28),
   ('Aug', 39),
   ('Sep', 78),
   ('Oct', 46),
   ('Nov', 42),
   ('Dec', 97)])]

In [83]:
ts_months.map(lambda x: x[2][4][1]).take(1)

[100]

In [88]:
ts_months_vector = ts_months.map(lambda x: (x[0],( x[1], [x[2][i][1]for i in range(12)])))

In [90]:
ts_months_vector.take(5)

[('onedirection',
  (('Museums', 'Indianapolis', 'Indiana'),
   [84, 87, 90, 86, 100, 54, 84, 29, 5, 85, 13, 1])),
 ('fortwanechildrenzoo',
  (('Zoos', 'Fort Wayne', 'Indiana'),
   [16, 56, 49, 87, 51, 66, 28, 39, 78, 46, 42, 97])),
 ('basilicaofthedacredheart',
  (('Church', 'South Bend', 'Indiana'),
   [38, 79, 101, 24, 56, 98, 38, 37, 87, 14, 66, 88])),
 ('universityofnotredame',
  (('University', 'South Bend', 'Indiana'),
   [26, 27, 25, 6, 62, 55, 49, 62, 3, 38, 20, 70])),
 ('parkviewfield',
  (('Arenas', 'Fort Wayne', 'Indiana'),
   [23, 43, 4, 77, 15, 56, 86, 32, 99, 86, 16, 27]))]